In [1]:
# coding: UTF-8
import argparse
import logging
import os
import random
import sys
from datetime import datetime
from posixpath import join
os.environ['CUDA_VISIBLE_DEVICES']='1'
sys.path.append('..')
import faiss
import numpy as np
import pandas as pd
# using labse
# from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data
from torch.nn import *

from loader.DBP15KRawNeighbors import DBP15KRawNeighbors
from script.preprocess.deal_raw_dataset import MyRawdataset
from settings import *

sys.argv=['']
del sys

### Trainer

1. link_loader:  
mode:选择子数据集

In [2]:
link = {}
mode = 'zh_en'
f = 'test.ref'
link_data = pd.read_csv(join(join(DATA_DIR, 'DBP15K', mode), f), sep='\t', header=None)
link_data.columns = ['entity1', 'entity2']
entity1_id = link_data['entity1'].values.tolist()
entity2_id = link_data['entity2'].values.tolist()
for i, _ in enumerate(entity1_id):
    link[entity1_id[i]] = entity2_id[i]
    link[entity2_id[i]] = entity1_id[i]

link_data['entity1']: pd column  
link_data['entity1'].values: np.array  
link{}: 保存的是zh->en和en->zh的双倍link  

In [3]:
len(link)

21000

if training:
self._model.update(self.model)之后  
会执行self._model.eval() 

2. save_model
torch.save(model.state_dict(), path) 保存模型，但是这里传入的是model? torch.save也可以保存model

3. evaluate
with torch.no_grad():
model.eval() 这两句取消了所有梯度计算和反向传播

for idx, item in enumerate(dataloader) dataloader返回的是idx和具体的数据

idx->[19388, 1]
x->[19388, 20, 788]
y->[19388, 1]

In [4]:
from model.layers_LaBSE_neighbor import *
parser = argparse.ArgumentParser()
args = parse_options(parser)
model = MyEmbedder(args, VOCAB_SIZE)
loader1 = DBP15KRawNeighbors(args.language, "1")
myset1 = MyRawdataset(loader1.id_neighbors_dict, loader1.id_adj_tensor_dict)
eval_loader1 = Data.DataLoader(
            dataset=myset1,  # torch TensorDataset format
            batch_size=args.batch_size,  # all test data
            shuffle=True,
            drop_last=False,
        )

In [5]:
model.device

device(type='cuda', index=0)

In [6]:
ids_1, ids_2, vector_1, vector_2 = list(), list(), list(), list()
inverse_ids_2 = dict()
val = next(iter(eval_loader1))
batch_in = val[0][:,:,:768]
center = batch_in[:,0]
center_neigh = batch_in
adj = batch_in[:,:,768:]       

In [7]:
bs,n = center_neigh.size()[:2]
print(bs,n)
h = center_neigh.unsqueeze(1)
h.shape

64 20


torch.Size([64, 1, 20, 768])

In [8]:
w = torch.nn.Parameter(torch.Tensor(12,768,768))

In [9]:
nn.init.xavier_uniform_(w)

Parameter containing:
tensor([[[ 1.7958e-03, -4.4733e-04, -3.0534e-03,  ...,  1.6334e-03,
           1.7520e-03, -4.5796e-04],
         [ 2.6854e-04,  3.1771e-04, -1.4820e-03,  ...,  2.1904e-03,
          -2.3089e-03, -1.0791e-03],
         [ 1.2658e-04,  1.6283e-03,  2.1464e-03,  ..., -2.2390e-03,
           1.0469e-03, -2.2112e-03],
         ...,
         [ 1.6108e-03, -2.0411e-03, -1.7799e-03,  ...,  6.3695e-04,
           2.2237e-03,  2.3011e-03],
         [ 7.2261e-04,  2.0061e-03, -1.0557e-03,  ...,  1.6976e-03,
          -1.5535e-03,  1.9487e-03],
         [ 3.6737e-04, -7.1399e-04, -2.0122e-03,  ...,  1.3452e-03,
          -1.9306e-03, -2.1280e-03]],

        [[ 1.1023e-04, -3.4434e-04, -2.3686e-03,  ...,  7.7027e-04,
           3.0684e-03,  6.0066e-04],
         [ 8.4089e-04,  1.8310e-03,  7.3043e-04,  ..., -3.0405e-03,
          -5.2540e-05,  3.8311e-04],
         [ 3.1565e-03, -1.0865e-03,  1.3456e-03,  ..., -9.5849e-04,
          -8.7422e-04, -2.4232e-03],
         ...,
   

In [10]:
h_prime = torch.matmul(h,w)

In [11]:
h_prime.shape

torch.Size([64, 12, 20, 768])

In [12]:
a_src = Parameter(torch.Tensor(12, 768, 1))
attn_src = torch.matmul(torch.tanh(h_prime), a_src)
attn_src.shape

torch.Size([64, 12, 20, 1])

In [13]:
attn = attn_src.expand(-1, -1, -1, n)
leak = nn.LeakyReLU(negative_slope=0.2)
attn = leak(attn)
output = torch.matmul(attn, h_prime)
output.shape

torch.Size([64, 12, 20, 768])

In [14]:
center_neigh = output.squeeze(1)
center_neigh = center_neigh[:, 0]
center_neigh.shape

torch.Size([64, 20, 768])

这里center_neigh就是输出的entity_vector

In [15]:
center_neigh = F.normalize(center_neigh, p=2, dim=1)
###entity_vector
center_neigh.shape

torch.Size([64, 20, 768])

In [16]:
ids_1, ids_2, vector_1, vector_2 = list(), list(), list(), list()
inverse_ids_2 = dict()
id_data_1 = val[1]
print(id_data_1.shape)
ids_1.extend(id_data_1.squeeze().tolist())
print(torch.tensor(ids_1).shape)
print(len(ids_1))

torch.Size([64, 1])
torch.Size([64])
64


In [17]:
def link_loader(mode, valid=False):
    link = {}
    if valid == False:
        f = 'test.ref'
    else:
        f = 'valid.ref'
    link_data = pd.read_csv(join(join(DATA_DIR, 'DBP15K', mode), f), sep='\t', header=None)
    link_data.columns = ['entity1', 'entity2']
    entity1_id = link_data['entity1'].values.tolist()
    entity2_id = link_data['entity2'].values.tolist()
    for i, _ in enumerate(entity1_id):
        link[entity1_id[i]] = entity2_id[i]
        link[entity2_id[i]] = entity1_id[i]
    return link

In [18]:
link = link_loader(args.language)

source = [_id for _id in ids_1 if _id in link]
source

[23461,
 22146,
 22117,
 9110,
 6327,
 24516,
 21476,
 21320,
 7647,
 6112,
 5992,
 5524,
 8695,
 10173,
 23048,
 7937,
 7205,
 21747,
 22626,
 23375,
 7323,
 8107,
 24077,
 24825,
 9434,
 7726,
 22110,
 10223,
 5226]

In [19]:
for idx, _id in enumerate(ids_1):
    inverse_ids_2[_id] = idx

In [46]:
target = np.array([inverse_ids_2[link[_id]] if link[_id] in inverse_ids_2 else 99999 for _id in source])
print(target)
print(target.shape)

[99999 99999 99999 99999 99999 99999 99999 99999 99999 99999 99999 99999
 99999 99999 99999 99999 99999 99999 99999 99999 99999 99999 99999 99999
 99999 99999 99999 99999 99999]
(29,)


In [66]:
test = np.array([10,20,30,40,50,60,999,999,999])
print(test.shape)
I = np.array([10,10,10,1000,10,40,10,10,10])
hit = (I == test[:,np.newaxis]).astype(np.int32).sum()
print(hit)
map1 = (I == test[:,np.newaxis])
print(map1)
print(map1.shape)

(9,)
8
[[ True  True  True False  True False  True  True  True]
 [False False False False False False False False False]
 [False False False False False False False False False]
 [False False False False False  True False False False]
 [False False False False False False False False False]
 [False False False False False False False False False]
 [False False False False False False False False False]
 [False False False False False False False False False]
 [False False False False False False False False False]]
[[False False False  True False  True False False False]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True False  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True]
 [ True  True  True  Tr

In [21]:
inverse_ids_2

{528: 0,
 23461: 1,
 22146: 2,
 22117: 3,
 9110: 4,
 28535: 5,
 6327: 6,
 24516: 7,
 529: 8,
 21476: 9,
 21320: 10,
 7647: 11,
 27224: 12,
 6112: 13,
 26865: 14,
 5992: 15,
 5524: 16,
 4433: 17,
 8695: 18,
 10173: 19,
 2357: 20,
 2756: 21,
 1298: 22,
 23048: 23,
 7937: 24,
 27697: 25,
 1161: 26,
 7205: 27,
 27293: 28,
 25755: 29,
 21747: 30,
 4018: 31,
 2312: 32,
 1740: 33,
 22626: 34,
 901: 35,
 23375: 36,
 26200: 37,
 1691: 38,
 29239: 39,
 4157: 40,
 3094: 41,
 7323: 42,
 1726: 43,
 8107: 44,
 24077: 45,
 28141: 46,
 24825: 47,
 2976: 48,
 26250: 49,
 1320: 50,
 738: 51,
 29218: 52,
 3745: 53,
 28123: 54,
 9434: 55,
 7726: 56,
 22110: 57,
 29853: 58,
 4216: 59,
 2563: 60,
 10223: 61,
 5226: 62,
 28585: 63}

In [22]:
src_idx = [idx for idx in range(len(ids_1)) if ids_1[idx] in link]
print(len(src_idx)==len(source))

True


In [23]:
# v1是一个batch
v1 = np.concatenate(tuple(center_neigh.squeeze().detach().cpu().numpy()), axis=0)[src_idx,:]
v1.shape

(29, 768)

In [24]:
index = faiss.IndexFlatL2(v1.shape[1])
#是768维上找相似

In [25]:
index.add(np.ascontiguousarray(v1))
index.ntotal
#一个batch64个数据，1280个triple

29

In [50]:
D, I = index.search(np.ascontiguousarray(v1), 10)
hit1 = (I[:, 0] == target).astype(np.int32).sum() / len(source)
hit1
print(I[:,0].shape)
print(type(I))

(29,)
<class 'numpy.ndarray'>
